### 1. Install Packages

In [1]:
# Install necessary packages
%pip install facebook-business
%pip install psycopg2

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### 2. Import the Necessary Libraries

In [2]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy import types
from sqlalchemy import text

### 3. Using the Meta Marketing API to Retrieve the Facebook/Instagram ads data

In [1]:
from facebook_business.adobjects.adaccount import AdAccount
from facebook_business.adobjects.adsinsights import AdsInsights
from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.adreportrun import AdReportRun
import time


access_token = 'EAAOhxmNeaWYBOy1mGgvm2AJAVtRASENjRjL7IC3xXi7QDFvwXg7oZBtEomxpHDq269SWb7w54lt6AEPMWwU3wlneuXVV4SGUrZCfw6vFkXZABOJbcbpt1z8PcA2PJR481fJwIZC6ZCVGDacTA3WKMQ4eMts7udhTr6Kd9ClILxWgFZCSqszNNFKYeLOHAZD'
ad_account_id = 'act_382451670462161'
app_secret = '27924ee55b8a48add1bf57360b91ed7c'
app_id = '1022298372860262'

# Initialize the Facebook API
FacebookAdsApi.init(access_token=access_token)

ad_account = AdAccount(ad_account_id)

# Define minimal fields to retrieve
fields = [
    'campaign_name',
    'adset_name',
    'ad_name',
    'cpm',
    'impressions',
    'reach',
    'video_play_actions',
    'video_p95_watched_actions',
    'video_avg_time_watched_actions',
    'clicks',
    'ctr',
    'outbound_clicks_ctr',
    'actions',
    'cpc',
    'cost_per_action_type',
    'spend',
    'purchase_roas',
    'action_values',
    'frequency',

]


# Define parameters
params = {
    'time_range': {'since':'2023-01-01','until':'2024-08-06'},
    # 'date_preset': 'yesterday',
    'level': 'ad',  # Choose the level: campaign, adset, or ad
    'time_increment': 1,  # Obtener datos diarios
    'action_report_time': 'conversion',
    'breakdowns': ['age', 'gender']
}

# Create and start an async job to import high volume data
async_job = ad_account.get_insights_async(params=params, fields=fields)
job_id = async_job[AdReportRun.Field.id]
print(f"Job ID: {job_id}")



Job ID: 879686480848867


### 4. Check Job Status

In [4]:
# Function to check the status of the job
def check_job_status(job_id):
    async_job = AdReportRun(job_id)
    while True:
        job_status = async_job.api_get(fields=[AdReportRun.Field.async_status])[AdReportRun.Field.async_status]
        print(f"Job Status: {job_status}")
        if job_status == 'Job Completed':
            break
        elif job_status == 'Job Failed':
            raise Exception("Job Failed")
        time.sleep(10)  # Wait for 10 seconds before checking again

check_job_status(job_id)


Job Status: Job Not Started
Job Status: Job Started
Job Status: Job Started
Job Status: Job Started
Job Status: Job Started
Job Status: Job Started
Job Status: Job Started
Job Status: Job Completed


### 5. Retrieve and Process Data

In [5]:
# Retrieve the results once the job is complete
def get_insights_data(job_id):
    async_job = AdReportRun(job_id)
    insights = async_job.get_insights()
    data = [insight.export_all_data() for insight in insights]
    return pd.DataFrame(data)

df = get_insights_data(job_id)


### 6. Extract Specific Values from Nested Fields

In [6]:
# Function to extract specific values from nested fields
def extract_specific_value(nested_field, value_type):
    if nested_field is not None:
        if isinstance(nested_field, list):
            for item in nested_field:
                if item['action_type'] == value_type:
                    return item['value']
        elif isinstance(nested_field, dict):
            if nested_field['action_type'] == value_type:
                return nested_field['value']
    return 0

# Extract specific values and create new columns
df['video_play_actions'] = df['video_play_actions'].apply(lambda x: extract_specific_value(x, 'video_view'))
df['video_p95_watched_actions'] = df['video_p95_watched_actions'].apply(lambda x: extract_specific_value(x, 'video_view'))
df['video_avg_time_watched_actions'] = df['video_avg_time_watched_actions'].apply(lambda x: extract_specific_value(x, 'video_view'))
df['outbound_clicks_ctr'] = df['outbound_clicks_ctr'].apply(lambda x: extract_specific_value(x, 'outbound_click'))
df['link_clicks'] = df['actions'].apply(lambda x: extract_specific_value(x, 'link_click'))
df['view_content'] = df['actions'].apply(lambda x: extract_specific_value(x, 'view_content'))
df['video_views_3_Sec'] = df['actions'].apply(lambda x: extract_specific_value(x, 'video_view'))
df['Add_to_cart'] = df['actions'].apply(lambda x: extract_specific_value(x, 'offsite_conversion.fb_pixel_add_to_cart'))
df['initiate_checkout'] = df['actions'].apply(lambda x: extract_specific_value(x, 'offsite_conversion.fb_pixel_initiate_checkout'))
df['purchase'] = df['actions'].apply(lambda x: extract_specific_value(x, 'offsite_conversion.fb_pixel_purchase'))
df['link_click_cost'] = df['cost_per_action_type'].apply(lambda x: extract_specific_value(x, 'link_click'))
df['view_content_cost'] = df['cost_per_action_type'].apply(lambda x: extract_specific_value(x, 'view_content'))
df['add_to_cart_cost'] = df['cost_per_action_type'].apply(lambda x: extract_specific_value(x, 'add_to_cart'))
df['purchase_cost'] = df['cost_per_action_type'].apply(lambda x: extract_specific_value(x, 'purchase'))
df['conversion_value'] = df['action_values'].apply(lambda x: extract_specific_value(x, 'offsite_conversion.fb_pixel_purchase'))
df['ROAS'] = df['purchase_roas'].apply(lambda x: extract_specific_value(x, 'omni_purchase'))

# Save to CSV
df.to_csv('facebook_ads_data.csv', index=False)
print("Data retrieved and saved to 'facebook_ads_data.csv'")


Data retrieved and saved to 'facebook_ads_data.csv'


### 7. Select relevant columns

In [9]:
# Select relevant columns
columns_to_keep = [
        'date_start', 'date_stop', 'campaign_name', 'adset_name', 'ad_name', 
        'cpm', 'impressions', 'reach', 'video_play_actions', 'video_views_3_Sec', 'video_p95_watched_actions', 
        'video_avg_time_watched_actions','clicks', 'link_clicks', 'ctr', 'outbound_clicks_ctr', 'view_content', 'Add_to_cart', 'initiate_checkout', 'purchase', 'cpc', 'link_click_cost', 'view_content_cost', 'add_to_cart_cost', 'purchase_cost', 'spend', 'ROAS', 'conversion_value', 'age', 'gender', 'frequency'
]

df = df[columns_to_keep]

pd.set_option('display.max_columns', None) # Display all columns
df.head() # Display the first few rows of the DataFrame


,date_start,date_stop,campaign_name,adset_name,ad_name,cpm,impressions,reach,video_play_actions,video_views_3_Sec,video_p95_watched_actions,video_avg_time_watched_actions,clicks,link_clicks,ctr,outbound_clicks_ctr,view_content,Add_to_cart,initiate_checkout,purchase,cpc,link_click_cost,view_content_cost,add_to_cart_cost,purchase_cost,spend,ROAS,conversion_value,age,gender,frequency
0,2024-06-01,2024-06-01,PM-Warm-Remarketing-05-03-234,Remarketing,Vídeo Retargeting 1,56.558317,6,6,5,1,0,2,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0.33935,0,0,18-24,female,1
1,2024-06-01,2024-06-01,PM-Warm-Remarketing-05-03-234,Remarketing,Vídeo Retargeting 1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,18-24,male,1
2,2024-06-01,2024-06-01,PM-Warm-Remarketing-05-03-234,Remarketing,Vídeo Retargeting 1,9.98088,1,1,1,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0.009981,0,0,18-24,unknown,1
3,2024-06-01,2024-06-01,PM-Warm-Remarketing-05-03-234,Remarketing,Vídeo Retargeting 1,21.122326,43,37,33,7,3,7,1,1,2.325581,2.325581,1,0,0,0,0.90826,0.90826,0.90826,0,0,0.90826,0,0,25-34,female,1.162162
4,2024-06-01,2024-06-01,PM-Warm-Remarketing-05-03-234,Remarketing,Vídeo Retargeting 1,9.98088,1,1,1,0,0,1,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0.009981,0,0,25-34,male,1


### 8. Data cleaning (Remove duplicates)

In [10]:
#Remove any duplicates in case there are some.
df = df.drop_duplicates() 
df.head()

,date_start,date_stop,campaign_name,adset_name,ad_name,cpm,impressions,reach,video_play_actions,video_views_3_Sec,video_p95_watched_actions,video_avg_time_watched_actions,clicks,link_clicks,ctr,outbound_clicks_ctr,view_content,Add_to_cart,initiate_checkout,purchase,cpc,link_click_cost,view_content_cost,add_to_cart_cost,purchase_cost,spend,ROAS,conversion_value,age,gender,frequency
0,2024-06-01,2024-06-01,PM-Warm-Remarketing-05-03-234,Remarketing,Vídeo Retargeting 1,56.558317,6,6,5,1,0,2,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0.33935,0,0,18-24,female,1
1,2024-06-01,2024-06-01,PM-Warm-Remarketing-05-03-234,Remarketing,Vídeo Retargeting 1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,18-24,male,1
2,2024-06-01,2024-06-01,PM-Warm-Remarketing-05-03-234,Remarketing,Vídeo Retargeting 1,9.98088,1,1,1,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0.009981,0,0,18-24,unknown,1
3,2024-06-01,2024-06-01,PM-Warm-Remarketing-05-03-234,Remarketing,Vídeo Retargeting 1,21.122326,43,37,33,7,3,7,1,1,2.325581,2.325581,1,0,0,0,0.90826,0.90826,0.90826,0,0,0.90826,0,0,25-34,female,1.162162
4,2024-06-01,2024-06-01,PM-Warm-Remarketing-05-03-234,Remarketing,Vídeo Retargeting 1,9.98088,1,1,1,0,0,1,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0.009981,0,0,25-34,male,1


### 9. Data cleaning (Find null values)

In [11]:
# Get column names and corresponding count of null values
null_columns = {}
for col in df.columns:
    null_count = df[col].isnull().sum()
    if null_count > 0:
        null_columns[col] = null_count
        
print("Columns with null values and their counts:")
print(null_columns)        

Columns with null values and their counts:
{'cpm': 57, 'ctr': 57, 'cpc': 6523}


### 10. Data cleaning (Convert null values to 0)

In [12]:
#Convert null values to 0

df = df.fillna(0)
#Check
null_columns = {}
for col in df.columns:
    null_count = df[col].isnull().sum()
    if null_count > 0:
        null_columns[col] = null_count
        
print("Columns with null values and their counts:")
print(null_columns)

Columns with null values and their counts:
{}


### 11. Data cleaning (Check grammar in columns)

In [13]:
#Correct grammar in the ad_name column
df['ad_name'] = df['ad_name'].replace('(?i)video', 'Vídeo', regex=True)
df.head()

,date_start,date_stop,campaign_name,adset_name,ad_name,cpm,impressions,reach,video_play_actions,video_views_3_Sec,video_p95_watched_actions,video_avg_time_watched_actions,clicks,link_clicks,ctr,outbound_clicks_ctr,view_content,Add_to_cart,initiate_checkout,purchase,cpc,link_click_cost,view_content_cost,add_to_cart_cost,purchase_cost,spend,ROAS,conversion_value,age,gender,frequency
0,2024-06-01,2024-06-01,PM-Warm-Remarketing-05-03-234,Remarketing,Vídeo Retargeting 1,56.558317,6,6,5,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0.33935,0,0,18-24,female,1
1,2024-06-01,2024-06-01,PM-Warm-Remarketing-05-03-234,Remarketing,Vídeo Retargeting 1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,18-24,male,1
2,2024-06-01,2024-06-01,PM-Warm-Remarketing-05-03-234,Remarketing,Vídeo Retargeting 1,9.98088,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.009981,0,0,18-24,unknown,1
3,2024-06-01,2024-06-01,PM-Warm-Remarketing-05-03-234,Remarketing,Vídeo Retargeting 1,21.122326,43,37,33,7,3,7,1,1,2.325581,2.325581,1,0,0,0,0.90826,0.90826,0.90826,0,0,0.90826,0,0,25-34,female,1.162162
4,2024-06-01,2024-06-01,PM-Warm-Remarketing-05-03-234,Remarketing,Vídeo Retargeting 1,9.98088,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.009981,0,0,25-34,male,1


### 12. Save data into a CSV

In [14]:
# Save to CSV
df.to_csv('facebook_ads_data_clean.csv', header= df.columns, index=False, encoding= 'utf-8-sig')
print("Data retrieved and saved to 'facebook_ads_data_clean.csv")

Data retrieved and saved to 'facebook_ads_data_clean.csv


### 13. Export data to a PostgreSQL Database

In [15]:
#Connect to PostgreSQL and Insert Data

# PostgreSQL database credentials
db_user = 'Gio'
db_password = 'Gio4everful.'
db_host = 'localhost'  # or your database host
db_port = '5432'  # or your database port
db_name = 'Fb ads Acala'

# Define mapping of DataFrame columns to PostgreSQL data types
dtype_mapping = {
    'date_start': types.Date,
    'date_stop': types.Date,
    'campaign_name': types.String(length=255),
    'adset_name': types.String(length=255),
    'ad_name': types.String(length=255),
    'cpm': types.Float(precision=2),
    'impressions': types.Integer,
    'reach': types.Integer,
    'video_play_actions': types.Integer,
    'video_views_3_Sec': types.Integer,
    'video_p95_watched_actions': types.Integer,
    'clicks': types.Integer,
    'link_clicks': types.Integer,
    'video_avg_time_watched_actions': types.Float(precision=2),
    'ctr': types.Float(precision=4),
    'outbound_clicks_ctr': types.Float(precision=4),
    'view_content': types.Integer,
    'Add_to_cart': types.Integer,
    'initiate_checkout': types.Integer,
    'purchase': types.Integer,
    'cpc': types.Float(precision=2),
    'link_click_cost': types.Float(precision=2),
    'view_content_cost': types.Float(precision=2),
    'add_to_cart_cost': types.Float(precision=2),
    'purchase_cost': types.Float(precision=2),
    'spend': types.Float(precision=2),
    'ROAS': types.Float(precision=2),
    'conversion_value': types.Float(precision=2),
    'age': types.String(length=50),  
    'gender': types.String(length=10),
    'frequency': types.Float(precision=2)
}

# Create SQLAlchemy engine to connect to PostgreSQL database
engine = create_engine(f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

# Connect to PostgreSQL and delete existing data, comment this block when you want to actually append new data and not deleting the actual.
with engine.connect() as connection:
    try:
        delete_statement = text("DELETE FROM facebook_ads_data;")
        result = connection.execute(delete_statement)
        print(f"Deleted {result.rowcount} rows from facebook_ads_data.")
        
        # Explicitly commit the transaction
        connection.commit()
        
        print("Transaction committed.")
    except Exception as e:
        print(f"Error during delete operation: {str(e)}")
        # Rollback in case of error
        connection.rollback()

# Save DataFrame to PostgreSQL
try:
    df.to_sql('facebook_ads_data', engine, if_exists='append', index=False, dtype=dtype_mapping)
    print("Data saved to PostgreSQL database successfully!")
except Exception as e:
    print(f"Error saving data to PostgreSQL: {str(e)}")

Deleted 11015 rows from facebook_ads_data.
Transaction committed.
Data saved to PostgreSQL database successfully!
